## Costa Rican Household Poverty Level Prediction

A quick look at both the training and test data shows that three columns have lots of missing values: 

- monthly rent (v2a1) 
- number of tablets (v18q1)  
- years behind in school (rez_esc)

If missing values are estimated by grouping based on socio-economic status ('Target' column), a near perfect classification is obtained (this "cheat" case is shown at the end). Obviously this is not useful because 'Target' is not given for the test set, but it reveals the need for a proper estimation of those columns. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('../input/train.csv')
df.replace('yes',1,inplace=True)
df.replace('no', 0, inplace=True)
df.sort_values('Target', inplace=True)

In [ ]:
cols = df.columns.values

In [ ]:
y = df.iloc[:,-1].values

In [ ]:
# Function that extracts the X matrix from dataframe 'data'. 
# It first attempts to fill the missing data using household averages.
# If not available, it will use population averages. 
# This is run after the prediction models for the three columns with many missing values.
def get_X_remove_nan(data, train = 1):
    cols = data.columns.values
    hog_ind=np.where(cols=='idhogar')[0][0]
    #X_raw keeps the idhorag 
    if train == 1:
        X_raw = data.iloc[:, :-1].values # exclude target for training data
    else:
        X_raw = data.values # no target for test data     
    # XX throws away the id (at index = 0) and idhorag
    XX = np.delete(X_raw, hog_ind, axis=1)[:,1:]
    XX = XX.astype(float)
    _ , n = XX.shape
    households = data['idhogar'].unique()
    # first attempt to replace nan's with values from same households, when at least one available
    for hh in households:
        aux = XX[X_raw[:,hog_ind] == hh]
        k, _ = aux.shape
        for i in range(n):
            if all(map(lambda x: x!=x, aux[:,i])):
                pass
            else:
                not_nan_ind = [m for m in range(k)]
                nan_ind = []
                for j in range(k):
                    if aux[j,i] != aux[j,i]:
                        nan_ind.append(j)
                        not_nan_ind.remove(j)
                aux_mean = np.average(aux[not_nan_ind, i])
                aux[nan_ind, i] = aux_mean
        XX[X_raw[:,hog_ind] == hh] = aux
    
    # one last pass to replace remaining nan's with means values with those in the same socio-economic class
    imputer = SimpleImputer(strategy = 'mean')
    imputer = imputer.fit(XX)
    XX = imputer.transform(XX)
    return XX

### Filling in the missing values for 3 columns using predictive models based on the other 137 columns

For each of the three columns with lots of missing data, I make a regression model to predict the missing data based on the values of the other 137 columns (treating the column under consideration as target and excluding the other two columns). Linear regression, SVR, and random forest are attempted, and random forest performs best. 

In [ ]:
def collect_existing_values(colsX, s, df, train = 1): 
    # separates nan's and existing values in column s and retunrs two dataframes
    # also returns the X and y for rows where column s has data
    # and X for rows where column s is missing
    # colX is list of columns with two of the three problematic columns removed (s is kept)
    dfX = df[colsX]
    df_exist = dfX[dfX[s] == dfX[s]] # rows where value of the column s is not nan
    df_nans = dfX[dfX[s] != dfX[s]] # rows where the value of column s is nan
    y = df_exist[s].values
    colsX.remove(s)
    X = get_X_remove_nan(df_exist[colsX], train)
    X_nans = get_X_remove_nan(df_nans[colsX], train) # X for rows where value of column s is nan
    return X, y, df_exist, df_nans, X_nans

In [ ]:
process_X = Pipeline([('sc', StandardScaler())]) 

In [ ]:
reg1 = Pipeline([('forest', RandomForestRegressor())])
reg2 = Pipeline([('svr', SVR(kernel = 'rbf', gamma='scale'))])
reg3 = Pipeline([('linear', LinearRegression())])
regs = [reg1, reg2, reg3]

In [ ]:
params = [{'forest__n_estimators': [20, 40, 80]}, {}, {}]

In [ ]:
sweep_to_reg = {0:0, 1:0, 2:0, 3:1, 4:2}
model_name = {0: 'Random Forest', 1: 'SVR', 2: 'Linear Regression'}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grids = []
for i in range(3):
    grids.append(GridSearchCV(regs[i], param_grid=params[i]))

####  1- Predicting monthly rent (v2a1)

In [ ]:
cols_v2a1 = [i for i in cols if i not in ['v18q1','rez_esc']] # has 'v2a1' but not the other two
Xv2, yv2, dfv2_exist, dfv2_nans, Xv2_nans = collect_existing_values(cols_v2a1, 'v2a1', df)

In [ ]:
Xv2_train, Xv2_val, yv2_train, yv2_val = train_test_split(Xv2, yv2, test_size = 0.2)

In [ ]:
Xv2_train = process_X.fit_transform(Xv2_train)
Xv2_val = process_X.transform(Xv2_val)
Xv2_nans = process_X.transform(Xv2_nans)

In [ ]:
scores = np.array([])
for i in range(3):
    grids[i].fit(Xv2_train, yv2_train)
    scores = np.concatenate((scores, grids[i].cv_results_['mean_test_score']))

In [ ]:
model_selected = sweep_to_reg[np.argmax(scores)]
reg_v2a1 = grids[model_selected].best_estimator_
model_name[model_selected]

In [ ]:
yv2_pred = reg_v2a1.predict(Xv2_val)
RMSE = np.sqrt(mean_squared_error(yv2_val, yv2_pred))
RMSE = RMSE / np.average(yv2_val)
print("Normalized RMSE is", RMSE)

In [ ]:
yv2_nans = reg_v2a1.predict(Xv2_nans)

#### 2- Predicting the number of tablets (v18q1)

In [ ]:
cols_v18 = [i for i in cols if i not in ['v2a1','rez_esc']] # has 'v18q1' but not the other two
Xv18, yv18, dfv18_exist, dfv18_nans, Xv18_nans = collect_existing_values(cols_v18, 'v18q1', df)

In [ ]:
Xv18_train, Xv18_val, yv18_train, yv18_val = train_test_split(Xv18, yv18, test_size = 0.2)

In [ ]:
Xv18_train = process_X.fit_transform(Xv18_train)
Xv18_val = process_X.transform(Xv18_val)
Xv18_nans = process_X.transform(Xv18_nans)

In [ ]:
scores = np.array([])
for i in range(3):
    grids[i].fit(Xv18_train, yv18_train)
    scores = np.concatenate((scores, grids[i].cv_results_['mean_test_score']))

In [ ]:
model_selected = sweep_to_reg[np.argmax(scores)]
reg_v18q1 = grids[model_selected].best_estimator_
model_name[model_selected]

In [ ]:
yv18_pred = reg_v18q1.predict(Xv18_val)
RMSE = np.sqrt(mean_squared_error(yv18_val, yv18_pred))
RMSE = RMSE / np.average(yv18_val)
print("Normalized RMSE is", RMSE)

In [ ]:
yv18_nans = reg_v18q1.predict(Xv18_nans)

#### 3- Predicting Years behind in school (rez_esc)

In [ ]:
cols_rez = [i for i in cols if i not in ['v2a1','v18q1']] # has 'rez_esc' but not the other two
Xrez, yrez, dfrez_exist, dfrez_nans, Xrez_nans = collect_existing_values(cols_rez, 'rez_esc', df)

In [ ]:
Xrez_train, Xrez_val, yrez_train, yrez_val = train_test_split(Xrez, yrez, test_size = 0.2)

In [ ]:
Xrez_train = process_X.fit_transform(Xrez_train)
Xrez_val = process_X.transform(Xrez_val)
Xrez_nans = process_X.transform(Xrez_nans)

In [ ]:
scores = np.array([])
for i in range(3):
    grids[i].fit(Xrez_train, yrez_train)
    scores = np.concatenate((scores, grids[i].cv_results_['mean_test_score']))

In [ ]:
model_selected = sweep_to_reg[np.argmax(scores)]
reg_rez_esc = grids[model_selected].best_estimator_
model_name[model_selected]

In [ ]:
yrez_pred = reg_rez_esc.predict(Xrez_val)
RMSE = np.sqrt(mean_squared_error(yrez_val, yrez_pred))
RMSE = RMSE / np.average(yrez_val)
print("Normalized RMSE is", RMSE)

In [ ]:
yrez_nans = reg_rez_esc.predict(Xrez_nans)

### Replacing the missing values with the predicted ones

In [ ]:
df2 = df.copy()

In [ ]:
df2.loc[dfv2_nans.index, 'v2a1'] = yv2_nans
df2.loc[dfv18_nans.index, 'v18q1'] = yv18_nans
df2.loc[dfrez_nans.index, 'rez_esc'] = yrez_nans

In [ ]:
y = df2.iloc[:,-1].values

In [ ]:
X = get_X_remove_nan(df2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train = process_X.fit_transform(X_train)
X_val = process_X.transform(X_val)

### Classification for training data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifiers = Pipeline([('forest', RandomForestClassifier(criterion = 'entropy'))])
params = [{'forest': [RandomForestClassifier(criterion= 'entropy')], 
          'forest__n_estimators' : [10, 20, 40, 80, 160]}]
grid = GridSearchCV(classifiers, cv = 5, param_grid= params)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.cv_results_['mean_test_score']

In [ ]:
selected_classifier = grid.best_estimator_

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = selected_classifier.predict(X_val)
cm = confusion_matrix(y_val, y_pred)
cm

In [ ]:
print("Classification rate is", np.sum(y_val==y_pred) / float(len(y_val)))
err_1 = (np.sum(cm[0])-cm[0,0])/float(np.sum(cm[0]))
print("Misclassification rate for group 1 is", err_1)

### Predicting the test set Targets

In [ ]:
df_test = pd.read_csv('../input/test.csv')

In [ ]:
df_test.replace('yes',1,inplace=True)
df_test.replace('no', 0, inplace=True)

#### Predicting and replacing the missing values

In [ ]:
cols_v2a1 = [i for i in cols if i not in ['v18q1','rez_esc']] # has 'v2a1' but not the other two
cols_v2a1.remove('Target')
_, _, _, dfv2_nans_test, Xv2_nans_test = collect_existing_values(cols_v2a1, 'v2a1', df_test, train=0)

In [ ]:
yv2_nans_test = reg_v2a1.predict(Xv2_nans_test)

In [ ]:
cols_v18 = [i for i in cols if i not in ['v2a1','rez_esc']] # has 'v18q1' but not the other two
cols_v18.remove('Target')
_, _, _, dfv18_nans_test, Xv18_nans_test = collect_existing_values(cols_v18, 'v18q1', df_test, train=0)

In [ ]:
yv18_nans_test = reg_v18q1.predict(Xv18_nans_test)

In [ ]:
cols_rez = [i for i in cols if i not in ['v2a1','v18q1']] # has 'rez_esc' but not the other two
cols_rez.remove('Target')
_, _, _, dfrez_nans_test, Xrez_nans_test = collect_existing_values(cols_rez, 'rez_esc', df_test, train=0)

In [ ]:
yrez_nans_test = reg_rez_esc.predict(Xrez_nans_test)

In [ ]:
df_test.loc[dfv2_nans_test.index, 'v2a1'] = yv2_nans_test
df_test.loc[dfv18_nans_test.index, 'v18q1'] = yv18_nans_test
df_test.loc[dfrez_nans_test.index, 'rez_esc'] = yrez_nans_test

In [ ]:
X_test = get_X_remove_nan(df_test, train = 0)

In [ ]:
X_test = process_X.transform(X_test)

In [ ]:
y_test = selected_classifier.predict(X_test)

In [ ]:
# putting results in a dataframe
df_result = pd.concat([df_test[['Id','idhogar']], pd.DataFrame({'Target': y_test})], axis = 1)
# household average
df_hh_ave = df_result.groupby('idhogar').mean() 
df_hh_ave = df_hh_ave.reset_index()
df_hh_ave['Target'] = np.round(df_hh_ave['Target'])
df_hh_ave = df_hh_ave.astype({'Target': int})
df_result = df_result.rename(columns={'Target': 'Target_individual'})
df_hh_ave = pd.merge(df_result, df_hh_ave, on='idhogar')

In [ ]:
df_hh_ave.index = df_hh_ave['Id']
df_final = df_hh_ave[['Target']]

In [ ]:
df_final.to_csv('submission_payamr.csv')

# Appendix

### Filling missing values based on the targets (cheat case)

In [ ]:
# This fills the missing values based on the values of 'Target'
targets = sorted(df['Target'].unique())
X = np.array([])
for t in targets:
    data = df[df['Target'] == t]
    XX = get_X_remove_nan(data)
    try:
        X = np.concatenate((X, XX), axis = 0)
    except:
        X = XX

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train = process_X.fit_transform(X_train)
X_val = process_X.transform(X_val)

In [ ]:
# Random Forest with 40 trees seemed to work just fine. 
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 40, criterion = 'entropy')
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_val)
cm = confusion_matrix(y_val, y_pred)

In [ ]:
cm

In [ ]:
print("Classification rate is", np.sum(y_val==y_pred) / float(len(y_val)))
err_1 = (np.sum(cm[0])-cm[0,0])/float(np.sum(cm[0]))
print("Misclassification rate for group 1 is", err_1)

(An artificially high classification rate achieved by using the targets in filling the missing values)